# Import Libraries

In [1]:
import Pkg;
# Pkg.add("ShiftedArrays")
# Pkg.add("TimeSeries")
# Pkg.add("DataFrames")
# Pkg.add("CSV")

include("CorwinSchultz.jl")
include("BekkerParkinsonVolatilityEstimator.jl")
using DataFrames, CSV, TimeSeries, Dates

# Read Data

In [2]:
ohlcvData = CSV.read("Data/AAPL_OHLCV_1m_Data.csv", DataFrames.DataFrame, header=1)
ohlcvData.Datetime = DateTime.(ohlcvData.Datetime, "yyyy-mm-dd HH:MM:SS")

ohlcvData |> size |> println

first(ohlcvData, 10)

(1949, 6)


,Datetime,Open,High,Low,Close,Volume
,DateTime,Float64,Float64,Float64,Float64,Int64
1,2022-08-22T09:30:00,169.69,169.71,168.93,169.52,3685094
2,2022-08-22T09:31:00,169.52,169.74,169.52,169.7,471038
3,2022-08-22T09:32:00,169.71,169.737,169.26,169.31,352459
4,2022-08-22T09:33:00,169.25,169.25,169.21,169.22,291817
5,2022-08-22T09:34:00,169.25,169.25,168.941,169.04,357077
6,2022-08-22T09:35:00,169.04,169.04,169.04,169.04,414994
7,2022-08-22T09:36:00,169.04,169.08,169.006,169.06,58095
8,2022-08-22T09:37:00,169.0,169.03,168.92,168.97,455048
9,2022-08-22T09:38:00,169.04,169.041,169.01,169.01,294641


# Corwin-Schultz Estimator

In [8]:
β = βEstimates(ohlcvData.High, ohlcvData.Low, 20);
γ = γEstimates(ohlcvData.High, ohlcvData.Low);
α = αEstimates(β, γ);


1949-element Vector{Union{Missing, Float64}}:
  missing
  missing
  missing
  missing
  missing
  missing
  missing
  missing
  missing
  missing
 ⋮
 0.0
 0.0007329125527335391
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0001576757509685724

In [4]:
corwinSchultz = corwinSchultzEstimator(ohlcvData.High, ohlcvData.Low, windowSpan=20)
corwinSchultzEstimatorTimeSeries = TimeArray((DateTime=ohlcvData.Datetime, Measure=corwinSchultz); timestamp=:DateTime)

1949×1 TimeArray{Union{Missing, Float64}, 2, DateTime, Matrix{Union{Missing, Float64}}} 2022-08-22T09:30:00 to 2022-08-26T16:00:00
│                     │ Measure │
├─────────────────────┼─────────┤
│ 2022-08-22T09:30:00 │ missing │
│ 2022-08-22T09:31:00 │ missing │
│ 2022-08-22T09:32:00 │ missing │
│ 2022-08-22T09:33:00 │ missing │
│ 2022-08-22T09:34:00 │ missing │
│ 2022-08-22T09:35:00 │ missing │
│ 2022-08-22T09:36:00 │ missing │
│ 2022-08-22T09:37:00 │ missing │
│ ⋮                   │ ⋮       │
│ 2022-08-26T15:54:00 │ -1.0    │
│ 2022-08-26T15:55:00 │ -1.0    │
│ 2022-08-26T15:56:00 │ -1.0    │
│ 2022-08-26T15:57:00 │ -1.0    │
│ 2022-08-26T15:58:00 │ -1.0    │
│ 2022-08-26T15:59:00 │ -1.0    │
│ 2022-08-26T16:00:00 │ -0.9998 │

# Bekker-Parkinson Volatility Estimator

In [10]:
σ = σEstimates(β, γ)

1949-element Vector{Union{Missing, Float64}}:
  missing
  missing
  missing
  missing
  missing
  missing
  missing
  missing
  missing
  missing
 ⋮
 0.006220021629354218
 0.0055198694646534634
 0.008632037106381593
 0.008896352588125614
 0.008742592370735727
 0.008908404237586938
 0.008118466760422507
 0.009085427523031701
 0.008312560636948029

In [6]:
bekkerParkinsonVolatility = bekkerParkinsonVolatilityEstimates(ohlcvData.High, ohlcvData.Low; windowSpan=20)
bekkerParkinsonVolatilityEstimatesTimeSeries = TimeArray((DateTime=ohlcvData.Datetime, Measure=bekkerParkinsonVolatility); timestamp=:DateTime)

1949×1 TimeArray{Union{Missing, Float64}, 2, DateTime, Matrix{Union{Missing, Float64}}} 2022-08-22T09:30:00 to 2022-08-26T16:00:00
│                     │ Measure │
├─────────────────────┼─────────┤
│ 2022-08-22T09:30:00 │ missing │
│ 2022-08-22T09:31:00 │ missing │
│ 2022-08-22T09:32:00 │ missing │
│ 2022-08-22T09:33:00 │ missing │
│ 2022-08-22T09:34:00 │ missing │
│ 2022-08-22T09:35:00 │ missing │
│ 2022-08-22T09:36:00 │ missing │
│ 2022-08-22T09:37:00 │ missing │
│ ⋮                   │ ⋮       │
│ 2022-08-26T15:54:00 │ 0.0086  │
│ 2022-08-26T15:55:00 │ 0.0089  │
│ 2022-08-26T15:56:00 │ 0.0087  │
│ 2022-08-26T15:57:00 │ 0.0089  │
│ 2022-08-26T15:58:00 │ 0.0081  │
│ 2022-08-26T15:59:00 │ 0.0091  │
│ 2022-08-26T16:00:00 │ 0.0083  │